# Table of GHG concentrations and ERF

In [1]:
import fair
from fair.forcing.ghg import meinshausen
import numpy as np
import scipy.stats as st
import pandas as pd

## Greenhouse gas concentrations

~~**TODO** what are we doing about +5% in non-CO2 gases?~~

In [2]:
ghg_obs = pd.read_excel('../data_input/LLGHG_history_AR6_v8a.xlsx', skiprows=22, sheet_name="MR_output", index_col=0)
for addyear in range(1751,1850):
    ghg_obs.loc[addyear, 'YYYY'] = np.nan
ghg_obs = ghg_obs.sort_index()
ghg_obs = ghg_obs.interpolate()
ghg_obs

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,...,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124,YYYY
YYYY,,,,,,,,,,,,,,,,,,,,,
1750,278.70,724.1000,270.0000,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1751,278.78,724.9350,270.0250,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1752,278.86,725.7700,270.0500,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1753,278.94,726.6050,270.0750,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1754,279.02,727.4400,270.1000,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,399.41,1833.8905,328.3025,83.4145,28.042,9.985158,18.0910,17.5655,6.5970,1.096239,...,0.11,0.0,0.424,0.066,0.62,1.05,0.39897,0.083,1.02,NaN
2016,402.86,1842.0015,329.1240,89.4825,28.950,11.972602,20.4815,19.1555,6.6600,1.205802,...,0.11,0.0,0.416,0.067,0.66,1.04,0.42501,0.083,1.02,NaN
2017,405.00,1849.2615,330.0400,95.7485,29.975,14.153317,23.2450,20.8340,6.8175,1.327504,...,0.11,0.0,0.416,0.067,0.66,1.04,0.43989,0.083,1.02,NaN


In [3]:
forcing_minor_1750 = pd.read_csv('../data_output/AR6_ERF_minorGHGs_1750-2019.csv', index_col=0)
list_signif_0005 = ['CO2', 'CH4', 'N2O'] + forcing_minor_1750.loc[:,forcing_minor_1750.loc[2019,:]>0.0005].columns.tolist()
list_signif_0010 = ['CO2', 'CH4', 'N2O'] + forcing_minor_1750.loc[:,forcing_minor_1750.loc[2019,:]>=0.001].columns.tolist()
print(list_signif_0005)
print(list_signif_0010)

['CO2', 'CH4', 'N2O', 'HFC-134a', 'HFC-23', 'HFC-32', 'HFC-125', 'HFC-143a', 'HFC-152a', 'HFC-245fa', 'SF6', 'SO2F2', 'CF4', 'C2F6', 'c-C4F8', 'CFC-12', 'CFC-11', 'CFC-113', 'CFC-114', 'CFC-115', 'CFC-13', 'HCFC-22', 'HCFC-141b', 'HCFC-142b', 'CCl4', 'CH2Cl2', 'Halon-1211', 'Halon-1301']
['CO2', 'CH4', 'N2O', 'HFC-134a', 'HFC-23', 'HFC-32', 'HFC-125', 'HFC-143a', 'SF6', 'CF4', 'C2F6', 'CFC-12', 'CFC-11', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b', 'CCl4']


In [4]:
list_signif = [   # from table 2.3 - not the same as ERF > 0.001
    'CO2',
    'CH4',
    'N2O',
    'CFC-11',
    'CFC-12',
    'CFC-113',
    'CFC-114',
    'CFC-115',
    'CCl4',
    'Halon-1211',
    'Halon-1301',
    'HCFC-22',
    'HCFC-141b',
    'HCFC-142b',
    'HFC-23',
    'HFC-32',
    'HFC-125',
    'HFC-134a',
    'HFC-143a',
    'CF4',
    'C2F6',
    'SF6',
]

In [5]:
gases = ghg_obs.columns.to_list()
gases.remove('YYYY')

In [6]:
ghg_obs.loc[[2019, 2011, 1850, 1750],list_signif_0010].round(
    {
        'CO2':1,
        'CH4':0,
        'N2O':1,
        'CFC-11':0,
        'CFC-12':0,
        'CFC-113':1,
        'CFC-114':1,
        'CFC-115':2,
        'CCl4':1,
        'Halon-1211':2,
        'Halon-1301':2,
        'HCFC-22':0,
        'HCFC-141b':1,
        'HCFC-142b':1,
        'HFC-23':1,
        'HFC-32':1,
        'HFC-125':1,
        'HFC-134a':1,
        'HFC-143a':1,
        'CF4':1,
        'C2F6':2,
        'SF6':2,
    }
).T.style.format('{0:g}')

YYYY,2019,2011,1850,1750
CO2,409.9,390.4,286.7,278.7
CH4,1866,1803,808,724
N2O,332.1,324.4,272.5,270
HFC-134a,107.6,62.7,0,0
HFC-23,32.4,24.1,0,0
HFC-32,20,4.7,0,0
HFC-125,29.4,10.3,0,0
HFC-143a,24,12,0,0
SF6,9.95,7.3,0,0
CF4,85.5,79,34,34


In [7]:
forcing1750 = {}
co2base1750 = np.zeros(270)
ch4base1750 = np.zeros(270)
n2obase1750 = np.zeros(270)
forcing1850 = {}
co2base1850 = np.zeros(270)
ch4base1850 = np.zeros(270)
n2obase1850 = np.zeros(270)
c = np.array([ghg_obs['CO2'].values, ghg_obs['CH4'].values, ghg_obs['N2O'].values])
for i, year in enumerate(range(1750,2020)):
    co2base1750[i], ch4base1750[i], n2obase1750[i] = meinshausen(c[:,i], [ghg_obs.loc[1750,'CO2'], ghg_obs.loc[1750,'CH4'], ghg_obs.loc[1750,'N2O']], scale_F2x=False)
    co2base1850[i], ch4base1850[i], n2obase1850[i] = meinshausen(c[:,i], [ghg_obs.loc[1850,'CO2'], ghg_obs.loc[1850,'CH4'], ghg_obs.loc[1850,'N2O']], scale_F2x=False)

# include rapid adjustments for CO2 and CH4 (FOD numbers):
forcing1750['CO2'] = 1.05 * co2base1750
forcing1750['CH4'] = 0.86 * ch4base1750
forcing1750['N2O'] = 1.07 * n2obase1750
forcing1850['CO2'] = 1.05 * co2base1850
forcing1850['CH4'] = 0.86 * ch4base1850
forcing1850['N2O'] = 1.07 * n2obase1850

In [8]:
forcing1850['CO2'][269]

1.989129210693368

In [9]:
# These radiative efficiencies are from Oivind H
radeff = {
    'HFC-125':      0.23378,
    'HFC-134a':     0.16714,
    'HFC-143a':     0.168,
    'HFC-152a':     0.10174,
    'HFC-227ea':    0.27325,
    'HFC-23':       0.19111,
    'HFC-236fa':    0.25069,
    'HFC-245fa':    0.24498,
    'HFC-32':       0.11144,
    'HFC-365mfc':   0.22813,
    'HFC-43-10mee': 0.35731,
    'NF3':          0.20448,
    'C2F6':         0.26105,
    'C3F8':         0.26999,
    'n-C4F10':      0.36874,
    'n-C5F12':      0.4076,
    'n-C6F14':      0.44888,
    'i-C6F14':      0.44888,
    'C7F16':        0.50312,
    'C8F18':        0.55787,
    'CF4':          0.09859,
    'c-C4F8':       0.31392,
    'SF6':          0.56657,
    'SO2F2':        0.21074,
    'CCl4':         0.16616,
    'CFC-11':       0.25941,
    'CFC-112':      0.28192,
    'CFC-112a':     0.24564,
    'CFC-113':      0.30142,
    'CFC-113a':     0.24094, 
    'CFC-114':      0.31433,
    'CFC-114a':     0.29747,
    'CFC-115':      0.24625,
    'CFC-12':       0.31998,
    'CFC-13':       0.27752,
    'CH2Cl2':       0.02882,
    'CH3Br':        0.00432,
    'CH3CCl3':      0.06454,
    'CH3Cl':        0.00466,
    'CHCl3':        0.07357,
    'HCFC-124':     0.20721,
    'HCFC-133a':    0.14995,
    'HCFC-141b':    0.16065,
    'HCFC-142b':    0.19329,
    'HCFC-22':      0.21385,
    'HCFC-31':      0.068,
    'Halon-1211':   0.30014,
    'Halon-1301':   0.29943,
    'Halon-2402':   0.31169,
}

shortlived = {
    'HFC-125':      False,
    'HFC-134a':     False,
    'HFC-143a':     False,
    'HFC-152a':     True,
    'HFC-227ea':    False,
    'HFC-23':       False,
    'HFC-236fa':    False,
    'HFC-245fa':    False,
    'HFC-32':       False,
    'HFC-365mfc':   False,
    'HFC-43-10mee': False,
    'NF3':          False,
    'C2F6':         False,
    'C3F8':         False,
    'n-C4F10':      False,
    'n-C5F12':      False,
    'n-C6F14':      False,
    'i-C6F14':      False,
    'C7F16':        False,
    'C8F18':        False,
    'CF4':          False,
    'c-C4F8':       True,
    'SF6':          False,
    'SO2F2':        False,
    'CCl4':         False,
    'CFC-11':       False,
    'CFC-112':      False,
    'CFC-112a':     False,
    'CFC-113':      False,
    'CFC-113a':     False,
    'CFC-114':      False,
    'CFC-114a':     False,
    'CFC-115':      False,
    'CFC-12':       False,
    'CFC-13':       False,
    'CH2Cl2':       True,
    'CH3Br':        False,
    'CH3CCl3':      True,
    'CH3Cl':        True,
    'CHCl3':        True,
    'HCFC-124':     False,
    'HCFC-133a':    True,
    'HCFC-141b':    False,
    'HCFC-142b':    False,
    'HCFC-22':      False,
    'HCFC-31':      True,
    'Halon-1211':   False,
    'Halon-1301':   False,
    'Halon-2402':   False,
}

trop_adjustment_scale = radeff.copy()
for key in trop_adjustment_scale.keys():
    trop_adjustment_scale[key] = 1
trop_adjustment_scale['CFC-11'] = 1.13
trop_adjustment_scale['CFC-12'] = 1.12

for gas in gases[3:]:
    forcing1750[gas] = (ghg_obs.loc[:,gas] - ghg_obs.loc[1750,gas]).values * radeff[gas] * 0.001 * trop_adjustment_scale[gas]
    forcing1850[gas] = (ghg_obs.loc[:,gas] - ghg_obs.loc[1850,gas]).values * radeff[gas] * 0.001 * trop_adjustment_scale[gas]

In [10]:
forcing1750

{'CO2': array([0.00000000e+00, 1.57113335e-03, 3.14185151e-03, 4.71215472e-03,
        6.28204323e-03, 7.85151726e-03, 9.42057707e-03, 1.09892229e-02,
        1.25574550e-02, 1.41252735e-02, 1.56926788e-02, 1.72596711e-02,
        1.88262505e-02, 2.03924175e-02, 2.19581720e-02, 2.35235146e-02,
        2.50884452e-02, 2.66529643e-02, 2.82170720e-02, 2.97807686e-02,
        3.13440543e-02, 3.29069293e-02, 3.44693939e-02, 3.60314483e-02,
        3.75930928e-02, 3.91543276e-02, 4.07151529e-02, 4.22755690e-02,
        4.38355760e-02, 4.53951743e-02, 4.69543641e-02, 4.85131455e-02,
        5.00715189e-02, 5.16294845e-02, 5.31870425e-02, 5.47441931e-02,
        5.63009366e-02, 5.78572731e-02, 5.94132031e-02, 6.09687266e-02,
        6.25238439e-02, 6.40785552e-02, 6.56328608e-02, 6.71867609e-02,
        6.87402558e-02, 7.02933456e-02, 7.18460306e-02, 7.33983110e-02,
        7.49501871e-02, 7.65016591e-02, 7.80527272e-02, 7.96033917e-02,
        8.11536528e-02, 8.27035106e-02, 8.42529656e-02, 8

In [11]:
gases

['CO2',
 'CH4',
 'N2O',
 'HFC-134a',
 'HFC-23',
 'HFC-32',
 'HFC-125',
 'HFC-143a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-236fa',
 'HFC-245fa',
 'HFC-365mfc',
 'HFC-43-10mee',
 'NF3',
 'SF6',
 'SO2F2',
 'CF4',
 'C2F6',
 'C3F8',
 'c-C4F8',
 'CFC-12',
 'CFC-11',
 'CFC-113',
 'CFC-114',
 'CFC-115',
 'CFC-13',
 'HCFC-22',
 'HCFC-141b',
 'HCFC-142b',
 'CH3CCl3',
 'CCl4',
 'CH3Cl',
 'CH3Br',
 'CH2Cl2',
 'CHCl3',
 'Halon-1211',
 'Halon-1301',
 'Halon-2402',
 'n-C4F10',
 'n-C5F12',
 'n-C6F14',
 'i-C6F14',
 'C7F16',
 'C8F18',
 'CFC-112',
 'CFC-112a',
 'CFC-113a',
 'CFC-114a',
 'HCFC-133a',
 'HCFC-31',
 'HCFC-124']

In [12]:
uncert = {}
uncert['CO2'] = 0.12
uncert['CH4'] = 0.20
uncert['N2O'] = 0.15
uncert['Halogens'] = 0.19

In [13]:
gases_cfcs = ['CFC-12', 'CFC-11', 'CFC-113', 'CFC-114', 'CFC-115', 'CFC-13']
gases_hcfcs= ['HCFC-22', 'HCFC-141b', 'HCFC-142b', 'HCFC-133a', 'HCFC-31', 'HCFC-124']
gases_hfcs = ['HFC-134a', 'HFC-23', 'HFC-32', 'HFC-125', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-236fa', 'HFC-245fa', 'HFC-365mfc', 'HFC-43-10mee',]
gases_halo = gases[3:]

In [14]:
forcing1750['CFCs'] = np.zeros(270)
forcing1750['HCFCs'] = np.zeros(270)
forcing1750['HFCs'] = np.zeros(270)
forcing1750['Halogens'] = np.zeros(270)
forcing1850['CFCs'] = np.zeros(270)
forcing1850['HCFCs'] = np.zeros(270)
forcing1850['HFCs'] = np.zeros(270)
forcing1850['Halogens'] = np.zeros(270)
for gas in gases_cfcs:
    forcing1750['CFCs'] = forcing1750['CFCs'] + forcing1750[gas]
    forcing1850['CFCs'] = forcing1850['CFCs'] + forcing1750[gas]
for gas in gases_hcfcs:
    forcing1750['HCFCs'] = forcing1750['HCFCs'] + forcing1750[gas]
    forcing1850['HCFCs'] = forcing1850['HCFCs'] + forcing1750[gas]
for gas in gases_hfcs:
    forcing1750['HFCs'] = forcing1750['HFCs'] + forcing1750[gas]
    forcing1850['HFCs'] = forcing1850['HFCs'] + forcing1750[gas]
for gas in gases_halo:
    forcing1750['Halogens'] = forcing1750['Halogens'] + forcing1750[gas]
    forcing1850['Halogens'] = forcing1850['Halogens'] + forcing1750[gas]
forcing1750['Total'] = forcing1750['CO2'] + forcing1750['CH4'] + forcing1750['N2O'] + forcing1750['Halogens']
forcing1850['Total'] = forcing1850['CO2'] + forcing1850['CH4'] + forcing1850['N2O'] + forcing1850['Halogens']

In [15]:
forcing1750 = pd.DataFrame(forcing1750)
forcing1850 = pd.DataFrame(forcing1850)

In [16]:
list_signif_plus = list_signif_0010 + ['CFCs', 'HCFCs', 'HFCs', 'Halogens', 'Total']

In [17]:
forcing1750.loc[[269,261],list_signif_plus].round(
    {
        'CO2': 3,
        'CH4': 3,
        'N2O': 3,
        'CFC-11': 3,
        'CFC-12': 3,
        'CFC-113': 3,
        'CFC-114': 3,
        'CFC-115': 3,
        'Halon-1211': 3,
        'Halon-1301': 3,
        'HCFC-22': 3,
        'HCFC-141b': 3,
        'HCFC-142b': 3,
        'HFC-23': 3,
        'HFC-32': 3,
        'HFC-125': 3,
        'HFC-134a': 3,
        'HFC-143a': 3,
        'HFC-152a': 3,
        'SF6': 3,
        'SO2F2': 3,
        'NF3': 3,
        'CF4': 3,
        'C2F6': 3,
        'CH3CCl3': 3,
        'CCl4': 3,
        'CFCs': 3,
        'HCFCs': 3,
        'HFCs': 3,
        'Halogens': 3,
        'Total': 3
    }
).T.style.format('{0:g}')

,269,261
CO2,2.149,1.873
CH4,0.547,0.524
N2O,0.209,0.185
HFC-134a,0.018,0.01
HFC-23,0.006,0.005
HFC-32,0.002,0.001
HFC-125,0.007,0.002
HFC-143a,0.004,0.002
SF6,0.006,0.004
CF4,0.005,0.004


In [18]:
forcing1850.loc[[269,261],list_signif_plus].round(
    {
        'CO2': 3,
        'CH4': 3,
        'N2O': 3,
        'CFC-11': 3,
        'CFC-12': 3,
        'CFC-113': 3,
        'CFC-114': 3,
        'CFC-115': 3,
        'Halon-1211': 3,
        'Halon-1301': 3,
        'HCFC-22': 3,
        'HCFC-141b': 3,
        'HCFC-142b': 3,
        'HFC-23': 3,
        'HFC-32': 3,
        'HFC-125': 3,
        'HFC-134a': 3,
        'HFC-143a': 3,
        'HFC-152a': 3,
        'SF6': 3,
        'SO2F2': 3,
        'NF3': 3,
        'CF4': 3,
        'C2F6': 3,
        'CH3CCl3': 3,
        'CCl4': 3,
        'CFCs': 3,
        'HCFCs': 3,
        'HFCs': 3,
        'Halogens': 3,
        'Total': 3,
    }
).T.style.format('{0:g}')

,269,261
CO2,1.989,1.714
CH4,0.496,0.473
N2O,0.2,0.176
HFC-134a,0.018,0.01
HFC-23,0.006,0.005
HFC-32,0.002,0.001
HFC-125,0.007,0.002
HFC-143a,0.004,0.002
SF6,0.006,0.004
CF4,0.005,0.004


In [19]:
df = pd.concat(
    (
        ghg_obs.loc[[2019, 2011, 1850, 1750],list_signif_0010].T, 
        forcing1850.loc[[269,261],list_signif_plus].T,
        forcing1750.loc[[269,261],list_signif_plus].T
    ), axis=1
)
df.columns = ['conc 2019', 'conc 2011', 'conc 1850', 'conc 1750', 'ERF 1850-2019', 'ERF 1850-2011', 'ERF 1750-2019', 'ERF 1750-2011']
df.to_csv('../data_output/table_7.5.csv')

In [20]:
# We'll assume that uncertainty in halogens is 19%

print(df.loc['CO2', 'ERF 1750-2019'] * uncert['CO2'])
print(df.loc['CO2', 'ERF 1850-2019'] * uncert['CO2'])
print(df.loc['CH4', 'ERF 1750-2019'] * uncert['CH4'])
print(df.loc['CH4', 'ERF 1850-2019'] * uncert['CH4'])
print(df.loc['N2O', 'ERF 1750-2019'] * uncert['N2O'])
print(df.loc['N2O', 'ERF 1850-2019'] * uncert['N2O'])
print(df.loc['Halogens', 'ERF 1750-2019'] * uncert['Halogens'])
print(df.loc['Halogens', 'ERF 1850-2019'] * uncert['Halogens'])
print(np.sqrt((df.loc['CO2', 'ERF 1750-2019'] * uncert['CO2'])**2)+
             ((df.loc['CH4', 'ERF 1750-2019'] * uncert['CH4'])**2)+
             ((df.loc['N2O', 'ERF 1750-2019'] * uncert['N2O'])**2)+
             ((df.loc['Halogens', 'ERF 1750-2019'] * uncert['Halogens'])**2)
     )
print(np.sqrt((df.loc['CO2', 'ERF 1850-2019'] * uncert['CO2'])**2)+
             ((df.loc['CH4', 'ERF 1850-2019'] * uncert['CH4'])**2)+
             ((df.loc['N2O', 'ERF 1850-2019'] * uncert['N2O'])**2)+
             ((df.loc['Halogens', 'ERF 1850-2019'] * uncert['Halogens'])**2)
     )

0.2578630096335504
0.23869550528320416
0.10943387356392496
0.09929666727113927
0.031322445192765835
0.02999560166247822
0.07752169110228581
0.07752169110228581
0.27682949048096755
0.25546468212481177
